In [1]:
import sys
sys.path.append("..")
from data.preparer import load_news_dataset
from babble import Explanation
from babble import BabbleStream
from babble.Candidate import Candidate 

from metal.analysis import lf_summary
from metal.analysis import label_coverage
from metal import LabelModel
from metal.tuners import RandomSearchTuner
from babble.utils import ExplanationIO

import pandas as pd
from datetime import datetime
from snorkel.labeling import filter_unlabeled_dataframe

stat_history = pd.DataFrame()
import nltk
nltk.download("punkt")

import pandas as pd
from datetime import datetime
stat_history = pd.DataFrame()

[nltk_data] Downloading package punkt to /Users/sara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## The Data

These texts discuss either gun politics (1) or computer electronics (0).

If you're not sure about the correct label, that's fine -- either make your best guess or just skip the example.

In [2]:
# Unzip the data. (Don't worry about this, it should be already unzipped.)
# Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

#!unzip -P PASSWORD data/data.zip

Load the dataset into training, validation, development, and test sets

In [2]:
df_train, df_dev, df_valid, df_test = load_news_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

477 training examples
400 development examples
100 validation examples
100 test examples


/Users/sara/Documents/GitHub/snorkel-notebooks/venv/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
# this is a helper class to transform our data into a format Babble can parse
df_train["label"] = -2
dfs = [df_train, df_dev, df_test]

for df in dfs:
    df["id"] = range(len(df))

Cs = [df.apply(lambda x: Candidate(x), axis=1) for df in dfs]

# babble labble uses 1 and 2 for labels, while our data uses 0 and 1
# add 1 to convert
Ys = [df.label.values + 1 for df in dfs]


Define the labels for this task.

In [172]:
ABSTAIN = 0
ELECTRONICS = 1
GUNS = 2

# Babble Tutorial
## News forum classification

### You will work with a subset of the 20 NewsGroup dataset. 
The texts shown are from one of two forums:
 1. Computer Electronics (Label 1)
 2. Gun Politics Forum (Label 2)
Your job is to create a training data set to classify texts as belonging to one of these two forums.

You will do this by writing natural language explanations of why you would label an example a certain way (1 (ELECTRONICS), 2 (GUNS), or 0 (ABSTAIN or no label)).
These explanations will be parsed into functions which will be aggregated by Snorkel to create training data from unlabeled examples.

You can evaluate your progress based on the coverage and f1 score of your label model, or by training a logistic regression classifier on the data and evaluating the test result.

In [173]:
# start timer!
stat_history.append({"time": datetime.now(), "num_lfs": 0}, ignore_index=True)

,num_lfs,time
0,0.0,2020-02-01 16:32:14.032294


Load the data into a *BabbleStream*: an object that iteratively displays candidates, collects and parses explanations.

In [282]:
# aliases are a way to refer to a set of words in a rule.
aliases = {
    "units": ["joules", "volts", "ohms", "MHz"]
}


babbler = BabbleStream(Cs, Ys, balanced=True, shuffled=True, seed=456, aliases=aliases)

Grammar construction complete.


In [283]:
def prettyprint(candidate):
    # just a helper function to print the candidate nicely
    print(candidate.text)

Let's look at an example candidate!

In [284]:
# Rerun this cell to get a new example
candidate = babbler.next()
prettyprint(candidate)

This is a followup post to something I've written previously.  Several
people responded with good information, but I don't think I communicated 
exactly what I am looking for.
	I'm working on a custom I/O device that will communicate with a host
via RS-232.  My custom circuitry will use an 80C186EB or EC CPU and require
about 64K of RAM (preferably FLASH RAM).  In looking around, I see that lots
of people have engineered UART-CPU-ROM-RAM subsystems that are ready to be
interfaced to your custom I/O devices.  It's been done so much, that it 
would be best if I can avoid reinventing a system.  It just needs to use
an 80C186 (or 188) CPU, and be able to load a program from the host then 
transfer control to that program.  Well, there's one other thing the ROM
needs to know how to do.  It should have routines to send and receive bytes
to/from the host, that utilize the hardware control lines (DTR,RTS,DTS,CTS).
Everything I've seen is in the $200.00 and up range.  That's too much for
this a

Next, we'll learn how to write a labelling function from a natural language explanation of why you chose a label for a given candidate.

## Create Explanations

Creating explanations generally happens in five steps:
1. View candidates
2. Write explanations
3. Get feedback
4. Update explanations 
5. Apply label aggregator

Steps 3-5 are optional; explanations may be submitted without any feedback on their quality. However, in our experience, observing how well explanations are being parsed and what their accuracy/coverage on a dev set are (if available) can quickly lead to simple improvements that yield significantly more useful labeling functions. Once a few labeling functions have been collected, you can use the label aggregator to identify candidates that are being mislabeled and write additional explanations targeting those failure modes.

Feel free to consult the internet or ask your experiment leader.

*(For the real task, you will be asked to write labeling functions, as quickly and accurately as possible. You will still be allowed to use the internet in this phase, but not ask your experiment leader.)*

### Collection

Use `babbler` to show candidates

In [285]:
candidate = babbler.next()
prettyprint(candidate)
print(candidate.mention_id)


I have the April 15, 1993 issue of the SF Chronicle in my lap.  Page
E7 (in the "Sporting Green" section) has a Trader's advert.  (The
copy is a bit screwed up - it says that the prices offered expire
4-14-93, but the ad is there.)

The SF Examiner and Chronicle run the same set of adverts (because
they have a joint printing/biz agreement and differ only in editorial
content).

I've seen gun ads recently in the merc, which is anti-gun editorially,
albeit not from traders, but from its competitors.

I don't know about the other papers.

Does Traders claim that things are changing?  When?


Because it's easier than telling the truth and no one much cares
either way.


Before you do, make sure that the bozos are actually doing what
you're accusing them of.

-andy
216


Is it about guns or electronics? What makes you think that? (If you don't know, it's okay to make your best guess or skip an example.)

Run the three examples given below, then parse them, and analyze them.
Then, you can try editing them and writing your own functions!

In [286]:
e0 = Explanation(
    # name of this rule, for your reference
    name='electr...', 
    # label to assign
    label=ELECTRONICS, 
    # natural language description of why you label the candidate this way
    condition='A word in the sentence starts with "electr"', 
)

In [287]:
e1 = Explanation(
    name = 'politics', 
    label = GUNS, 
    condition = 'Any of the words "election", "senator", "democrat", or "republican" are in the text', 
)

Below is an example of an explanation that uses an alias: "couples"

You can define more aliases where the BabbleStream is initialized.

In [288]:
e2 = Explanation(
    name = 'selfdefense', 
    label = GUNS, 
    condition = 'because the word "self" occurs before "defense"', 
)

In [289]:
e3 = Explanation(
    name = "e3", 
    label = ABSTAIN, 
    condition = '', 
    # candidate is an optional argument, it should be the id of an example labeled by this rule.
    # if the rule doesn't apply to the candidate you provide, it will be filtered!
    candidate = candidate
)

In [290]:
e4 = Explanation(
    name = "e4", 
    label = ABSTAIN, 
    condition = "", 
    candidate = candidate
)

Babble will parse your explanations into functions, then filter out functions that are duplicates, incorrectly label their given candidate, or assign the same label to all examples.

In [291]:
# Add any explanations that you haven't committed yet
explanations = [e0, e1, e2, e3]

parses, filtered = babbler.apply(explanations)

Building list of target candidate ids...
Collected 1 unique target candidate ids from 4 explanations.
Gathering desired candidates...
Could not find 1 target candidates with the following mention_ids (first 5):
{'text': '\nI have the April 15, 1993 issue of the SF Chronicle in my lap.  Page\nE7 (in the "Sporting Green" section) has a Trader\'s advert.  (The\ncopy is a bit screwed up - it says that the prices offered expire\n4-14-93, but the ad is there.)\n\nThe SF Examiner and Chronicle run the same set of adverts (because\nthey have a joint printing/biz agreement and differ only in editorial\ncontent).\n\nI\'ve seen gun ads recently in the merc, which is anti-gun editorially,\nalbeit not from traders, but from its competitors.\n\nI don\'t know about the other papers.\n\nDoes Traders claim that things are changing?  When?\n\n\nBecause it\'s easier than telling the truth and no one much cares\neither way.\n\n\nBefore you do, make sure that the bozos are actually doing what\nyou\'re accu

### Analysis
See how your parsed explanations performed

In [292]:
try: 
    babbler.analyze(parses)
except ValueError as e:
    print("It seems as though none of your labeling functions were parsed. See the cells above and below for more information.")
    print("ERROR:")
    print(e)

See which explanations were filtered and why

In [293]:
babbler.filtered_analysis(filtered)

SUMMARY
4 TOTAL:
1 Unparseable Explanation
1 Duplicate Semantics
0 Inconsistency with Example
1 Uniform Signature
1 Duplicate Signature
0 Lowest Coverage

[#1]: Unparseable Explanation

Explanation: 

Reason: This explanation couldn't be parsed.

Semantics: None


[#2]: Duplicate Semantics

Parse: return 1 if 'self'.(= 'defense') else 0

Reason: This parse is identical to one produced by the following explanation:
	"because the word "self" occurs before "defense""

Semantics: ('.root', ('.label', ('.int', 2), ('.call', ('.eq', ('.string', 'defense')), ('.string', 'self'))))


[#3]: Uniform Signature

Parse: return 1 if 'self'.(= 'defense') else 0

Reason: This parse labeled NONE of the 400 development examples

Semantics: ('.root', ('.label', ('.int', 2), ('.call', ('.eq', ('.string', 'defense')), ('.string', 'self'))))


[#4]: Duplicate Signature

Parse: return 1 if sum([s.startswith('electr') for s in [w for w in the word(s) the sentence]]).(>= 1) else 0

Reason: This parse labeled i

In [294]:
babbler.commit()

Added 2 parse(s) from 2 explanations to set. (Total # parses = 2)

Applying labeling functions to split 1
[========================================] 100%

Added 42 labels to split 1: L.nnz = 42, L.shape = (400, 2).
Applying labeling functions to split 2
[========================================] 100%

Added 9 labels to split 2: L.nnz = 9, L.shape = (100, 2).


### Evaluation
Get feedback on the performance of your explanations

In [295]:
Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], Ys[1], lf_names=lf_names)

Retrieved label matrix for split 0: L.nnz = 52, L.shape = (477, 2)
Retrieved label matrix for split 1: L.nnz = 42, L.shape = (400, 2)
Retrieved label matrix for split 2: L.nnz = 9, L.shape = (100, 2)


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
electr..._0,0,1,0.095,0.0025,0.0025,32,6,0.842105
politics_0,1,2,0.010,0.0025,0.0025,4,0,1.000000


In [296]:
search_space = {
    'n_epochs': [50, 100, 500],
    'lr': {'range': [0.01, 0.001], 'scale': 'log'},
    'show_plots': False,
}

tuner = RandomSearchTuner(LabelModel, seed=123)

label_aggregator = tuner.search(
    search_space, 
    train_args=[Ls[0]], 
    X_dev=Ls[1], Y_dev=Ys[1], 
    max_search=20, verbose=False, metric='f1')

# record statistics over time
pr, re, f1 = label_aggregator.score(Ls[1], Ys[1], metric=['precision', 'recall', 'f1'])
stats = {
    "precision": pr,
    "recall": re,
    "f1": f1,
    "time": datetime.now(),
    "training_label_coverage": label_coverage(Ls[0]),
    "training_label_size": label_coverage(Ls[0])*len(dfs[0])
}
stat_history = stat_history.append(stats, ignore_index=True)

[SUMMARY]
Best model: [2]
Best config: {'n_epochs': 100, 'show_plots': False, 'lr': 0.0011281415605346923, 'seed': 124}
Best score: 0.693069306930693
Precision: 0.530
Recall: 1.000
F1: 0.693


If you'd like to save the explanations you've generated, you can use the `ExplanationIO` object to write to or read them from file.

In [297]:
stat_history.to_csv("babbler_tutorial_statistics_history.csv")
FILE = "babbler_tutorial_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

AttributeError: 'NoneType' object has no attribute 'mention_id'

## Train Model
We can train a simple bag of words model on these labels, and see test accuracy.

(This step may take a while).

In [44]:
from analyzer import train_model

train_model(label_aggregator, df_train, df_valid, df_test, L_train)

NameError: name 'label_aggregator' is not defined